In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
import tensorflow as tf
from keras.layers import Dense, Conv1D, SpatialDropout1D, MaxPooling1D,Input, Flatten
from keras.layers.core import Lambda, Dropout
from keras import backend as K
from keras.regularizers import l2
from keras.models import Model
import os

from google.colab import drive
drive.mount("/content/gdrive")

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

nuc_arr = ['A','C','G','T']
#Function for calculating modified probability of splicing at SD1
def prob_SD1 (sd1_freq, sd2_freq):
    if (sd1_freq==0 and sd2_freq==0):
        return 0.0
    else:
        return sd1_freq/(sd1_freq+sd2_freq)
#Function converting nucleotide sequence to numerical array with 4 channels
def seq_to_arr (seq):
    seq_len = len(seq)
    arr_rep = np.zeros((seq_len, len(nuc_arr)))
    for i in range(seq_len):
        arr_rep[i][nuc_arr.index(seq[i])] = 1
    return arr_rep

#Storing model inputs and outputs
dataset_path = os.path.expanduser("/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/5SS_compressed.txt")
seq_len = 101
n = 265137
inputs = np.zeros((n,seq_len, 4))
prob_s1 = np.zeros(n)
with open(dataset_path) as f:
    ind = 0
    for line in f:
        mod_line = line.split('\t')
        inputs[ind] = seq_to_arr(mod_line[1])
        prob_s1[ind] = prob_SD1(float(mod_line[2]), float(mod_line[3][:-1]))
        ind += 1


In [0]:


def create_dropout_masks(dim, drop_prob):
    """Creates dropout masks that are fixed across all model inputs for use in estimation of Var(Y_{sample})
    Note: this function is specific to keras models used in this experiment; number of dropout masks J is deduced from dim.
    #Arguments
      dim: the numpy array shape of values entering the generated dropout masks for all points across all J fixed dropout masks
      drop_prob: the dropout probability
    #Returns
      dropout masks used in estimation of Var(Y_{sample})
    """
    if (len(dim)==4):
        return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3])), p=[drop_prob, 1-drop_prob])
    return 1/(1-drop_prob)*np.random.choice(2, size=((dim[0], 1, 1, dim[3], dim[4])), p=[drop_prob, 1-drop_prob])

def apply_dropout_masks(a,b):
    return np.squeeze(np.multiply(a,b))

def multi_mask_predict(layer_fn, multi_mask_input):
    """Applies Keras model layers to multiple arrays of layer inputs for all points across J fixed dropout masks
    #Arguments
        layer_fn: keras backend function applying the mapping corresponding to keras model layers
        multi_mask_input: numpy array of layer inputs for all points across J fixed dropout masks
    #Returns
        predictions from layer_fn applied to multi_mask_input as a numpy array
    """
    layer_output = []
    for mask_num in range(multi_mask_input.shape[0]):
        layer_output.append(layer_fn((multi_mask_input[mask_num], 1)))
        #layer_output.append(layer_fn([multi_mask_input[mask_num]]))
    return np.array(layer_output)


#Used for calculating test MSE and variance (average of MC dropout predictions)
def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean and variance of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
      f: function mapping model input and Keras backend learning_phase flag to model output
      x: input
      n_iter: number of repreated MC dropout predictions per point
    #Returns:
      Mean and variance of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0]))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])[:,0]
        result[i,:] = predictions
    prediction = result.mean(axis=0)
    uncertainty = result.var(axis=0)
    return prediction, uncertainty



In [0]:
#Active Learning Experiment Parameters/Settings
num_acquisitions = 1200
batch_size = 1
sample_size=5000
num_epochs = 300
acq_fn_dropout_iterations = 50
mse_dropout_iterations = 200
size_train = 75
#Training batch size (not active learning batch size)
batchsize=128
tau_inv_proportion = 0.15

#This is run for three values of ind:1, 2, and 3
ind = 1

In [0]:
#Loading Data
X = np.array(inputs)
y = np.array(prob_s1)

def initialize_model(curr_size_train):
  """Initializes a Keras model in each active learning iteration; enables model retraining from scratch in each iteration
  #Arguments:
    curr_size_train: current training size, which affects the weight decay
  #Returns:
    A new initialized Keras model
  """
  #Parameters
  dropout_prob = 0.15
  Weight_Decay = 0.025/curr_size_train
  #Model
  inputlayer = Input(shape=(seq_len, len(nuc_arr)))
  x = Conv1D(seq_len,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu')(inputlayer)
  x = MaxPooling1D(pool_size=3)(x)
  x = Conv1D(seq_len//2, (4), strides=1, activation='relu')(x)
  x = MaxPooling1D(pool_size=3)(x)
  x = Dropout(dropout_prob)(x)
  x = Flatten()(x)
  x = Dense(50, W_regularizer=l2(Weight_Decay), init='normal',  activation='relu')(x)
  x = Dropout(dropout_prob)(x)
  outputlayer = Dense(1, W_regularizer=l2(Weight_Decay), init='normal')(x)

  model = Model(inputlayer, outputlayer)
  model.compile(loss='mean_squared_error', optimizer='rmsprop')
  return model

#Note: code is currently set to *resume* active learning experiments. 
#To begin active learning experiments, several commented lines below should be uncommented and vice versa.

model_maxvar = initialize_model(size_train)

#Initializing datasets and filepaths
filepath_maxvar_mse = '/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Results_v3/Exp3_StartSize75_Wd0.025/MaxVar_BS'+str(batch_size)+'_MSE_Ind' + str(ind) + ".npy"
filepath_indices = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Results_v3/Exp3_StartSize75_Wd0.025/MaxVar_BS"+str(batch_size)+"_Indices_Ind" + str(ind) + ".npy"
#exp_mse_maxvar = []
exp_mse_maxvar = list(np.load(filepath_maxvar_mse))

all_train_indices = np.load('/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Scripts_v2/trainindices'+ str(ind) + ".npy")
#train_data_indices_maxvar = all_train_indices[0:size_train].tolist()
train_data_indices_maxvar = list(np.load(filepath_indices))
test_indices = np.load('/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/5p_Splicing/AL_Scripts_v2/testindices'+ str(ind) + ".npy")

X_train_maxvar = X[train_data_indices_maxvar]
y_train_maxvar = y[train_data_indices_maxvar]
X_test = X[test_indices]
y_test = y[test_indices]

#First iteration of active learning experiment
model_maxvar.fit(X_train_maxvar, y_train_maxvar, epochs=num_epochs, batch_size=batchsize, verbose=0)
f_maxvar = K.function([model_maxvar.layers[0].input, K.learning_phase()], 
               [model_maxvar.layers[-1].output])
predictions_with_uncertainty = predict_with_uncertainty(f_maxvar, X_test, n_iter=mse_dropout_iterations)
y_predicted = predictions_with_uncertainty[0]
mse = np.mean(np.square(y_test-y_predicted))
#Comment out line below when resuming an experiment
exp_mse_maxvar.append(mse)
print("Initial MSE: " + str(mse))
print('len mse: ' + str(len(exp_mse_maxvar)))
print('len indices: ' + str(len(train_data_indices_maxvar)))
num_acquisitions = num_acquisitions - batch_size*(len(exp_mse_maxvar)-1)
print('num acquisitions remaining: ' + str(num_acquisitions))

#Initializing pool indices
pool_indices_maxvar = [ind for ind in all_train_indices if ind not in train_data_indices_maxvar]
print('size train initial: ' + str(len(train_data_indices_maxvar)))
print('size pool initial: ' + str(len(pool_indices_maxvar)))

#Acquisition Loop
for acq in range(num_acquisitions//batch_size):
    #Max Var acquisition
    sample_indices = np.random.choice(pool_indices_maxvar, sample_size, replace=False)
    X_sample = X[sample_indices]
    predictions_with_uncertainty = predict_with_uncertainty(f_maxvar, X_sample, n_iter=acq_fn_dropout_iterations)
    pred_var = predictions_with_uncertainty[1]
    acq_ind_ind = np.flip(np.argsort(pred_var))[0:batch_size]
    all_acq_ind = sample_indices[acq_ind_ind]
    for acq_ind in all_acq_ind:
      train_data_indices_maxvar.append(acq_ind)
      pool_indices_maxvar.remove(acq_ind)
    X_train_maxvar = X[train_data_indices_maxvar]
    y_train_maxvar = y[train_data_indices_maxvar]
    #Retraining model and calculating MSE
    model_maxvar = initialize_model(len(train_data_indices_maxvar))
    model_maxvar.fit(X_train_maxvar, y_train_maxvar, epochs=num_epochs, batch_size=batchsize, verbose=0)
    f_maxvar = K.function([model_maxvar.layers[0].input, K.learning_phase()], 
               [model_maxvar.layers[-1].output])
    predictions_with_uncertainty = predict_with_uncertainty(f_maxvar, X_test, n_iter=mse_dropout_iterations)
    y_predicted = predictions_with_uncertainty[0]
    mse_maxvar = np.mean(np.square(y_test-y_predicted))
    exp_mse_maxvar.append(mse_maxvar)
    if (acq == num_acquisitions//batch_size -1) or acq % 5 == 0:
      np.save(filepath_maxvar_mse, np.array(exp_mse_maxvar))
      np.save(filepath_indices, np.array(train_data_indices_maxvar))
      #print(1)

    print('new train len: ' + str(len(train_data_indices_maxvar)))
    print('new pool len: ' + str(len(pool_indices_maxvar)))
    print('Number of complete acquisitions: ' + str(len(exp_mse_maxvar)) + ', latest MSE: ' + str(exp_mse_maxvar[-1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


Initial MSE: 0.096452423689346
len mse: 700
len indices: 773
num acquisitions remaining: 501
size train initial: 773
size pool initial: 237851


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


new train len: 774
new pool len: 237850
Number of complete acquisitions: 701, latest MSE: 0.09338815470866678


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


new train len: 891
new pool len: 237733
Number of complete acquisitions: 818, latest MSE: 0.0868910090974929


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


new train len: 892
new pool len: 237732
Number of complete acquisitions: 819, latest MSE: 0.09065685660066435


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
